In [1]:
import requests
env = open('../.env').readlines()
IS_PROD = True
env_prefix = "" if IS_PROD else "DEV_"
DISCORD_APPLICATION_ID = [x.split('=')[1].strip().replace('"', '') for x in env if x.startswith(f'{env_prefix}DISCORD_APPLICATION_ID')][0]
DISCORD_BOT_TOKEN = [x.split('=')[1].strip().replace('"', '') for x in env if x.startswith(f'{env_prefix}DISCORD_BOT_TOKEN')][0]
headers = {
    "Authorization": f"Bot {DISCORD_BOT_TOKEN}"
}
from typing import Optional
def get_command_id(name: str) -> Optional[str]:
    url = f"https://discord.com/api/v8/applications/{DISCORD_APPLICATION_ID}/commands"
    r = requests.get(url, headers=headers)
    d = r.json()
    results = [x.get("id") for x in d if x.get("name") == name]
    return results[0] if results else None

# LIST COMMANDS

In [15]:
url = f"https://discord.com/api/v8/applications/{DISCORD_APPLICATION_ID}/commands"
r = requests.get(url, headers=headers)
r.json()

[{'id': '920656302420684820',
  'application_id': '920612693864493066',
  'version': '937447797664403537',
  'default_permission': True,
  'default_member_permissions': None,
  'type': 1,
  'name': 'starter',
  'description': 'Generate a conversation starter given some topics',
  'dm_permission': None,
  'options': [{'type': 3,
    'name': 'topics',
    'description': "List of topics separated by ','. Example 'ice breaker,travel,dating'. (default: 'ice breaker')"},
   {'type': 3,
    'name': 'players',
    'description': "List of challenged players separated by ','. Example '@user1,@user2'"}]},
 {'id': '937361355932070030',
  'application_id': '920612693864493066',
  'version': '937361355932070031',
  'default_permission': True,
  'default_member_permissions': None,
  'type': 1,
  'name': 'about',
  'description': 'Give information about the bot',
  'dm_permission': None},
 {'id': '937372525892096011',
  'application_id': '920612693864493066',
  'version': '937459048939417600',
  'defa

# /starter

In [3]:
name = "starter"
d = {
    "name": name,
    "description": "Generate a conversation starter given some topics",
    "options": [
        {
            "name": "topics",
            "description": "List of topics separated by ','."
            + " Example 'ice breaker,travel,dating'. (default: 'ice breaker')",
            "type": 3,
            "required": False,
        },
        {
            "name": "players",
            "description": "List of challenged players separated by ','. Example '@user1,@user2'",
            "type": 3,
            "required": False,
        }
    ]
}
COMMAND_ID = get_command_id(name)
url = f"https://discord.com/api/v8/applications/{DISCORD_APPLICATION_ID}/commands/"+(f"/{COMMAND_ID}" if COMMAND_ID else "")
r =  requests.patch(url, headers=headers, json=d) if COMMAND_ID else requests.post(url, headers=headers, json=d)
r.json()

{'id': '912067168210931783',
 'application_id': '912000855186751489',
 'version': '937655825864998973',
 'default_permission': True,
 'default_member_permissions': None,
 'type': 1,
 'name': 'starter',
 'name_localizations': None,
 'description': 'Generate a conversation starter given some topics',
 'description_localizations': None,
 'dm_permission': None,
 'options': [{'type': 3,
   'name': 'topics',
   'name_localizations': None,
   'description': "List of topics separated by ','. Example 'ice breaker,travel,dating'. (default: 'ice breaker')",
   'description_localizations': None},
  {'type': 3,
   'name': 'players',
   'name_localizations': None,
   'description': "List of challenged players separated by ','. Example '@user1,@user2'",
   'description_localizations': None}]}

# COMMAND DELETE

In [13]:
name = ""
COMMAND_ID = get_command_id(name)
url = f"https://discord.com/api/v8/applications/{DISCORD_APPLICATION_ID}/commands/{COMMAND_ID}"
r = requests.delete(url, headers=headers)
r.text

''

# /about

Langame
A game where you can play with your friends.
How to play
Send this command to a channel:
/starter topics:ice breaker,travel,whatever topic your like

You can also add players to the Langame by adding the following option:
/starter players:@user1,@user2,@user3

With love, by https://langa.me/. Augmented conversations.


In [5]:
name = "about"
d = {
    "name": name,
    "description": "Give information about the bot",
}
COMMAND_ID = get_command_id(name)
url = f"https://discord.com/api/v8/applications/{DISCORD_APPLICATION_ID}/commands"+(f"/{COMMAND_ID}" if COMMAND_ID else "")
r =  requests.patch(url, headers=headers, json=d) if COMMAND_ID else requests.post(url, headers=headers, json=d)
r.json()

{'id': '937655936204562453',
 'application_id': '912000855186751489',
 'version': '937655936204562454',
 'default_permission': True,
 'default_member_permissions': None,
 'type': 1,
 'name': 'about',
 'name_localizations': None,
 'description': 'Give information about the bot',
 'description_localizations': None,
 'dm_permission': None}

# /sub

/sub topics:ice breaker,travel players:@louis030195,@bob27 frequency:12h

results in the bot posting conversation starters between these two every 12h on the given topics

Optionally use players:random to have the bot choose a random player from the server


### Execution

Topics:ice breaker,travel
Players:@louis030195,@bob27

What is your favorite place?

CTA component button to stop the sub, like button click on /unsub whatever

In [2]:
name = "sub"
d = {
    "name": name,
    "description": "Subscribe to frequent Langames",
    "options": [
        {
            "name": "topics",
            "description": "List of topics separated by ','. Example 'ice breaker,travel,dating'"
            + "(default 'ice breaker')",
            "type": 3,
            "required": False,
        },
        # how the players are selected?
        {
            "name": "players",
            "description": "How are the players selected on this server? (default: 'Random on this server')",
            "type": 3,
            "required": False,
            "choices": [
                {
                    "name": "Random on this server",
                    "value": "random_server"
                },
                {
                    "name": "Most talkative people on this channel",
                    "value": "talkative_channel"
                },
                # {
                #     "name": "Random on this channel",
                #     "value": "random_channel"
                # }
            ]
        },
        {
            "name": "frequency",
            "description": "Frequency in minutes,"
            + " hours or days. Example '30m' or '5h' or '2d'"
            + " (default is 3h)",
            "type": 3,
            "required": False,
        }
    ],
}
COMMAND_ID = get_command_id(name)
url = f"https://discord.com/api/v8/applications/{DISCORD_APPLICATION_ID}/commands"+(f"/{COMMAND_ID}" if COMMAND_ID else "")
r =  requests.patch(url, headers=headers, json=d) if COMMAND_ID else requests.post(url, headers=headers, json=d)
r.json()

{'id': '937656007562240060',
 'application_id': '912000855186751489',
 'version': '939257957747675196',
 'default_permission': True,
 'default_member_permissions': None,
 'type': 1,
 'name': 'sub',
 'name_localizations': None,
 'description': 'Subscribe to frequent Langames',
 'description_localizations': None,
 'dm_permission': None,
 'options': [{'type': 3,
   'name': 'topics',
   'name_localizations': None,
   'description': "List of topics separated by ','. Example 'ice breaker,travel,dating'(default 'ice breaker')",
   'description_localizations': None},
  {'type': 3,
   'name': 'players',
   'name_localizations': None,
   'description': "How are the players selected on this server? (default: 'Random on this server')",
   'description_localizations': None,
   'choices': [{'name': 'Random on this server',
     'value': 'random_server',
     'name_localizations': None},
    {'name': 'Most talkative people on this channel',
     'value': 'talkative_channel',
     'name_localizations'

# /unsub

Unsubscribe the server from Langames.

In [3]:
name = "unsub"
d = {
    "name": name,
    "description": "Unsubscribe from frequent Langames",
    # optionally can say "all" to unsubscribe from all channels
    "options": [
        {
            "name": "all",
            "description": "Unsubscribe Langames from all channels",
            "type": 3,
            "required": False,
            "choices": [
                {
                    "name": "Yes",
                    "value": "True",
                },
                {
                    "name": "No",
                    "value": "False",
                }
            ]
        },
    ],  
}
COMMAND_ID = get_command_id(name)
url = f"https://discord.com/api/v8/applications/{DISCORD_APPLICATION_ID}/commands"+(f"/{COMMAND_ID}" if COMMAND_ID else "")
r =  requests.patch(url, headers=headers, json=d) if COMMAND_ID else requests.post(url, headers=headers, json=d)
r.json()

{'id': '937656041238315028',
 'application_id': '912000855186751489',
 'version': '939257972993949818',
 'default_permission': True,
 'default_member_permissions': None,
 'type': 1,
 'name': 'unsub',
 'name_localizations': None,
 'description': 'Unsubscribe from frequent Langames',
 'description_localizations': None,
 'dm_permission': None,
 'options': [{'type': 3,
   'name': 'all',
   'name_localizations': None,
   'description': 'Unsubscribe Langames from all channels',
   'description_localizations': None,
   'choices': [{'name': 'Yes', 'value': 'True', 'name_localizations': None},
    {'name': 'No', 'value': 'False', 'name_localizations': None}]}]}

# Other

In [11]:
# url = f"https://discord.com/api/v8/guilds/888438975646285834/members/267955006974590976"
# url = f"https://discord.com/api/v8/guilds/888438975646285834/roles"
url = f"https://discord.com/api/v8/guilds/888438975646285834/members?limit=1000"
r = requests.get(url, headers=headers)
r.json()

[{'roles': ['933033976828948520', '933032577319731292'],
  'nick': None,
  'avatar': None,
  'premium_since': None,
  'joined_at': '2022-01-18T16:18:21.437000+00:00',
  'is_pending': False,
  'pending': False,
  'communication_disabled_until': None,
  'user': {'id': '172002275412279296',
   'username': 'Tatsu',
   'avatar': '43fc1879641f6239d6e4038eaa7dabbc',
   'discriminator': '8792',
   'public_flags': 589824,
   'bot': True},
  'mute': False,
  'deaf': False},
 {'roles': ['916740688396292216', '935092248167739432'],
  'nick': None,
  'avatar': None,
  'premium_since': None,
  'joined_at': '2021-09-17T14:59:18.242000+00:00',
  'is_pending': False,
  'pending': False,
  'communication_disabled_until': None,
  'user': {'id': '267955006974590976',
   'username': 'louis030195',
   'avatar': '1a8d658e1267a1b0b36f93c151e4c733',
   'discriminator': '2462',
   'public_flags': 0},
  'mute': False,
  'deaf': False},
 {'roles': ['933033976828948520', '935092248167739432'],
  'nick': None,
  'a

# Analysis

In [2]:
from langame import LangameClient
c = LangameClient("../config.yaml")
si = c._firestore_client.collection("social_interactions").order_by("guild_id").get()

In [10]:
guilds = {}
for e in si:
    g_id = e.get("guild_id")
    if g_id not in guilds:
        guilds[g_id] = set()
    guilds[g_id].add(e.get("channel_id"))
# turn into list
for g_id in guilds:
    guilds[g_id] = list(guilds[g_id])
guilds

{'937357295199846400': ['937357295199846403']}

In [13]:
for g_id in guilds:
    url = f"https://discord.com/api/v8/guilds/{g_id}"
    r =  requests.get(url, headers=headers)
    d = r.json()
    print("id:" + d.get("id", "has no id"))
    print("name:" + d.get("name", "has no name"))
    description = d.get("description", None)
    if description:
        print("description:" + description)
    print(d)

id:937357295199846400
name:test
{'id': '937357295199846400', 'name': 'test', 'icon': None, 'description': None, 'splash': None, 'discovery_splash': None, 'features': [], 'emojis': [], 'stickers': [], 'banner': None, 'owner_id': '267955006974590976', 'application_id': None, 'region': 'hongkong', 'afk_channel_id': None, 'afk_timeout': 300, 'system_channel_id': '937357295199846403', 'widget_enabled': False, 'widget_channel_id': None, 'verification_level': 0, 'roles': [{'id': '937357295199846400', 'name': '@everyone', 'permissions': '1071698660929', 'position': 0, 'color': 0, 'hoist': False, 'managed': False, 'mentionable': False, 'icon': None, 'unicode_emoji': None}, {'id': '937362052358484009', 'name': 'langame-alpha', 'permissions': '3088', 'position': 4, 'color': 0, 'hoist': False, 'managed': True, 'mentionable': False, 'icon': None, 'unicode_emoji': None, 'tags': {'bot_id': '920612693864493066'}}, {'id': '937652082314465310', 'name': 'Langamer', 'permissions': '1071698660929', 'positi

In [12]:
url = f"https://discord.com/api/v8/channels/937357295199846403"
r =  requests.get(url, headers=headers)
d = r.json()
d

{'id': '937357295199846403',
 'last_message_id': '939231061068615731',
 'type': 0,
 'name': 'général',
 'position': 0,
 'parent_id': '937357295199846401',
 'topic': None,
 'guild_id': '937357295199846400',
 'permission_overwrites': [],
 'nsfw': False,
 'rate_limit_per_user': 0}

In [56]:
url = f"https://discord.com/api/v8/channels/937357295199846403/messages"
r =  requests.get(url, headers=headers)
all_messages = r.json()
all_messages

[{'id': '939254349480214568',
  'type': 0,
  'content': 'Topics: ice breaker.\nPlayers: <@912000855186751489>,<@267955006974590976>.\n\n**“WHAT IF…?" :**',
  'channel_id': '937357295199846403',
  'author': {'id': '920612693864493066',
   'username': 'Langame-alpha',
   'avatar': '601bbe24bb4c6a29089e85e4113bb242',
   'discriminator': '5491',
   'public_flags': 0,
   'bot': True},
  'attachments': [],
  'embeds': [],
  'mentions': [{'id': '267955006974590976',
    'username': 'louis030195',
    'avatar': '1a8d658e1267a1b0b36f93c151e4c733',
    'discriminator': '2462',
    'public_flags': 0},
   {'id': '912000855186751489',
    'username': 'Langame',
    'avatar': '71c05f3e97fc8b6ebf015aad8e67186d',
    'discriminator': '8825',
    'public_flags': 524288,
    'bot': True}],
  'mention_roles': [],
  'pinned': False,
  'mention_everyone': False,
  'tts': False,
  'timestamp': '2022-02-04T20:21:27.418000+00:00',
  'edited_timestamp': None,
  'flags': 0,
  'components': []},
 {'id': '9392536

In [57]:
# find most talkative people on this channel
messages_per_user = {}
for e in all_messages:
    author_id = e.get("author").get("id")
    if author_id not in messages_per_user:
        messages_per_user[author_id] = []
    messages_per_user[author_id].append(e)
sorted_by_nb_messages = sorted(messages_per_user.items(), key=lambda x: len(x[1]), reverse=True)
# take top three distinct users
top_talkers = []
for e in sorted_by_nb_messages:
    if e[1][0].get("author").get("id") not in [j.get("id") for j in top_talkers]:
        top_talkers.append(e[1][0].get("author"))
top_talkers[:2]

[{'id': '920612693864493066',
  'username': 'Langame-alpha',
  'avatar': '601bbe24bb4c6a29089e85e4113bb242',
  'discriminator': '5491',
  'public_flags': 0,
  'bot': True},
 {'id': '912000855186751489',
  'username': 'Langame',
  'avatar': '71c05f3e97fc8b6ebf015aad8e67186d',
  'discriminator': '8825',
  'public_flags': 524288,
  'bot': True}]

In [46]:
sorted_by_nb_messages

[('912000855186751489',
  [{'id': '937718051342794792',
    'type': 20,
    'content': "Topics: backpacking.\n\n**what would you do with a year of your life where you could do all sorts of things (life stuff, not money stuff) that you haven't had yet - what would they be?**",
    'channel_id': '937357295199846403',
    'author': {'id': '912000855186751489',
     'username': 'Langame',
     'avatar': '71c05f3e97fc8b6ebf015aad8e67186d',
     'discriminator': '8825',
     'public_flags': 524288,
     'bot': True},
    'attachments': [],
    'embeds': [],
    'mentions': [],
    'mention_roles': [],
    'pinned': False,
    'mention_everyone': False,
    'tts': False,
    'timestamp': '2022-01-31T14:36:45.399000+00:00',
    'edited_timestamp': '2022-01-31T14:36:51.525000+00:00',
    'flags': 0,
    'components': [],
    'application_id': '912000855186751489',
    'interaction': {'id': '937718050147414026',
     'type': 2,
     'name': 'starter',
     'user': {'id': '267955006974590976',
  